Импорт необходимых библиотек

In [2]:
import pandas as pd
import numpy as np

import matplotlib.pyplot as plt

from catboost import CatBoostClassifier, cv, Pool

from sklearn.metrics import roc_auc_score
from sklearn.metrics import roc_curve
from sklearn.model_selection import StratifiedKFold

pd.options.display.max_columns=100

Загрузка обучающего и тестового датасетов

In [3]:
def do_cv(x_train, x_test, y_train, features, cat_features, params, splitter):
    oof_result = pd.Series([0] * x_train.shape[0], index=x_train.index)
    result = [0] * x_test.shape[0]
    scores = []
    for ind_trn, ind_test in splitter.split(x_train, y_train):
        X_train_df = x_train[features].iloc[ind_trn]
        X_valid_df = x_train[features].iloc[ind_test]
        y_train_df = y_train.iloc[ind_trn]
        y_valid_df = y_train.iloc[ind_test]
        
        eval_dataset = Pool(X_valid_df, y_valid_df, cat_features)
        model = CatBoostClassifier(**params)

        model.fit(
            X_train_df,  
            y_train_df,
            cat_features,
            eval_set=eval_dataset,
            verbose=0,
            use_best_model=True,
        )
        
        scores.append(roc_auc_score(y_valid_df, model.predict_proba(X_valid_df)[:, 1]))
        result += model.predict_proba(x_test[features])[:, 1:].flatten()/5
        oof_result.loc[X_valid_df.index] = model.predict_proba(X_valid_df)[:, 1]
        
    return scores, result, oof_result

In [4]:
train = pd.read_csv('data/catboost_train.csv')
test = pd.read_csv('data/catboost_test.csv')

In [5]:
train['emp_length'] = train['emp_length'].astype('int')
test['emp_length'] = test['emp_length'].astype('int')

In [6]:
targ_cols = [i for i in train.columns if i not in ['def', 'id'] ]

X_train = train[targ_cols]
X_test = test[targ_cols]

y_train = train["def"]

In [7]:
targ_cols

['installment',
 'dti',
 'funded_amnt',
 'annual_inc',
 'emp_length',
 'term',
 'inq_last_6mths',
 'mths_since_recent_inq',
 'delinq_2yrs',
 'chargeoff_within_12_mths',
 'num_accts_ever_120_pd',
 'num_tl_90g_dpd_24m',
 'acc_open_past_24mths',
 'avg_cur_bal',
 'tot_hi_cred_lim',
 'delinq_amnt',
 'purpose',
 'subgrade',
 'home_ownership',
 'psk',
 'percent',
 'percent_per_year',
 'residual_dti',
 'loan_income_ratio',
 'additional_dti',
 'dti_ratio_min',
 'dti_ratio_max',
 'waste_to_installment_ratio',
 'is_first_loan',
 'is_not_sber_client',
 'acc_open_past_24mths_ratio',
 'tot_hi_cred_lim_funded_amnt_ratio',
 'tot_hi_cred_lim_annual_inc_ratio',
 'even_salary',
 'even_installment',
 'is_emp_length_unknown',
 'is_dti_b_35',
 'is_common_amount',
 'funded_amnt_mod_1000',
 'funded_amnt_mod_500',
 'funded_amnt_mod_100',
 'funded_amnt_mod_10',
 'installment_log',
 'installment_sqrt',
 'dti_log',
 'dti_sqrt',
 'funded_amnt_log',
 'funded_amnt_sqrt',
 'annual_inc_log',
 'annual_inc_sqrt',
 'avg_

In [45]:
params = {
    "iterations": 1000,
#     "depth": 1,
#     "learning_rate": 0.08,
#     "l2_leaf_reg": 20, # 50 
    ""
    "loss_function": "Logloss",
    "verbose": False,
    "eval_metric": 'AUC',
    "od_type": 'Iter',
    "od_wait": 150,
#     "random_strength": 1,
}

init_cols = [
#     'percent',
#     'annual_inc',
#     'acc_open_past_24mths',
#     'installment',
#     'avg_cur_bal',
#     'mths_since_recent_inq',
#     'tot_hi_cred_lim',
#     'dti',
    
    'term',
    'acc_open_past_24mths', # 677 02
    'funded_amnt',
    'percent_per_year',
    'additional_dti', #671
    'num_accts_ever_120_pd',
    'annual_inc', # 682 024,
    'tot_hi_cred_lim',
    'purpose', # 6839 035
    'home_ownership', # 683 027
    
    'avg_cur_bal', # 684 041
    'mths_since_recent_inq',
    'installment',
    'funded_amnt_mod_100',
]

cat_features = [
    'purpose',
    'home_ownership',
]

rs = {}

skf = StratifiedKFold(n_splits=5, random_state=18, shuffle=True)
for feat in targ_cols:
    print(feat)
    if feat in init_cols:
        continue
        
    if train[feat].dtype == 'O':
        print('as_cat')
        scores, result, oof_result = do_cv(X_train, X_test, y_train, init_cols + [feat], cat_features + [feat], params, skf)
    else:
        scores, result, oof_result = do_cv(X_train, X_test, y_train, init_cols + [feat], cat_features, params, skf)
    print(feat, np.mean(scores), np.std(scores))
    rs[feat] = [np.mean(scores), np.std(scores)]

installment
dti
dti 0.6858255666316226 0.007062606054550438
funded_amnt
annual_inc
emp_length
emp_length 0.6867766331449788 0.008405391874692105
term
inq_last_6mths
inq_last_6mths 0.6863821369869756 0.006742062472084148
mths_since_recent_inq
delinq_2yrs
delinq_2yrs 0.6866868562374993 0.006974290408237733
chargeoff_within_12_mths
chargeoff_within_12_mths 0.6858111987663653 0.007422656516462929
num_accts_ever_120_pd
num_tl_90g_dpd_24m
num_tl_90g_dpd_24m 0.6866537711208649 0.007012233195139113
acc_open_past_24mths
avg_cur_bal
tot_hi_cred_lim
delinq_amnt
delinq_amnt 0.6864933157735973 0.006441567763371679
purpose
subgrade
as_cat
subgrade 0.6863912574924725 0.007974483352379467
home_ownership
psk
psk 0.6878471530598915 0.007112149494586045
percent
percent 0.6867703577115972 0.006548897787670939
percent_per_year
residual_dti
residual_dti 0.6858289655441907 0.007146898702614718
loan_income_ratio
loan_income_ratio 0.6857407527994148 0.006109776218752306
additional_dti
dti_ratio_min
dti_ratio_m

KeyboardInterrupt: 

In [33]:
pd.DataFrame(rs).T.sort_values(0).tail(10)

,0,1
installment_sqrt,0.686341,0.007611
tot_hi_cred_lim_funded_amnt_ratio,0.686343,0.006951
funded_amnt_mod_500,0.686694,0.007666
is_emp_length_unknown,0.686696,0.007782
funded_amnt_log,0.686803,0.007866
funded_amnt_sqrt,0.686804,0.007868
funded_amnt_mod_1000,0.686884,0.008411
tot_hi_cred_lim_log,0.687053,0.006785
tot_hi_cred_lim_sqrt,0.687053,0.006785
funded_amnt_mod_100,0.687893,0.006439


In [ ]:
pd.DataFrame(rs).T.sort_values(0).tail(10)

In [82]:
train

,installment,dti,funded_amnt,annual_inc,emp_length,term,inq_last_6mths,mths_since_recent_inq,delinq_2yrs,chargeoff_within_12_mths,num_accts_ever_120_pd,num_tl_90g_dpd_24m,acc_open_past_24mths,avg_cur_bal,tot_hi_cred_lim,delinq_amnt,def,id,purpose,subgrade,home_ownership,psk,percent,percent_per_year,residual_dti,loan_income_ratio,additional_dti,dti_ratio_min,dti_ratio_max,waste_to_installment_ratio,is_first_loan,is_not_sber_client,acc_open_past_24mths_ratio,tot_hi_cred_lim_funded_amnt_ratio,tot_hi_cred_lim_annual_inc_ratio,funded_amnt_tot_hi_cred_lim_ratio,annual_inc_tot_hi_cred_lim_ratio,avg_cur_bal_tot_hi_cred_lim_ratio,even_salary,even_installment,is_emp_length_unknown,is_dti_b_35,is_common_amount,funded_amnt_mod_1000,funded_amnt_mod_500,funded_amnt_mod_100,funded_amnt_mod_10,installment_log,installment_sqrt,dti_log,dti_sqrt,funded_amnt_log,funded_amnt_sqrt,annual_inc_log,annual_inc_sqrt,avg_cur_bal_log,avg_cur_bal_sqrt,tot_hi_cred_lim_log,tot_hi_cred_lim_sqrt
0,365.59,12.92,10500,66400.0,6.0,36,0,7.0,0,0,0.0,0.0,4.0,2879.0,35225.0,0,0,0,purpose_debt_consolidation,sub_grade_C2,home_ownership_RENT,13161.24,0.253451,8.4,6.312952,0.158133,19.527048,2.046586,0.661646,195.548748,False,False,0.222222,3.354762,0.530497,0.298084,1.885025,0.081732,0,0,0,0,0,500,0,0,0,5.901513,19.120408,2.558777,3.594440,9.259131,102.469508,11.103452,257.681975,7.965198,53.656314,10.469511,187.683244
1,535.54,19.11,15000,80000.0,10.0,36,1,2.0,1,0,3.0,0.0,4.0,13266.0,164512.0,0,0,1,purpose_credit_card,sub_grade_C5,home_ownership_MORTGAGE,19279.44,0.285296,9.5,11.076900,0.187500,27.143100,1.725212,0.704046,237.890727,False,False,0.173913,10.967467,2.056400,0.091179,0.486287,0.080638,1,0,0,0,1,0,0,0,0,6.283276,23.141737,2.950212,4.371499,9.615805,122.474487,11.289782,282.842712,9.492960,115.178123,12.010739,405.600789
2,517.16,26.55,22400,50000.0,10.0,60,4,2.0,0,0,0.0,0.0,3.0,12445.0,163277.0,0,0,2,purpose_credit_card,sub_grade_C1,home_ownership_MORTGAGE,31029.60,0.385250,7.7,14.138160,0.448000,38.961840,1.877896,0.681436,213.908655,False,False,0.130435,7.289152,3.265540,0.137190,0.306228,0.076220,1,0,0,0,0,400,400,0,0,6.248352,22.741152,3.279030,5.152669,10.016816,149.666295,10.819778,223.606798,9.429074,111.557160,12.003203,404.075488
3,342.66,5.74,9800,41000.0,0.0,36,1,4.0,1,0,0.0,0.0,5.0,1936.0,37600.0,0,0,3,purpose_debt_consolidation,sub_grade_C5,home_ownership_RENT,12335.76,0.258751,8.6,-4.289073,0.239024,15.769073,-1.338284,0.364004,57.233604,False,False,0.238095,3.836735,0.917073,0.260638,1.090426,0.051489,1,0,0,0,0,800,300,0,0,5.836739,18.511078,1.747461,2.395830,9.190138,98.994949,10.621327,202.484567,7.568379,44.000000,10.534759,193.907194
4,381.46,4.88,17325,74000.0,10.0,60,1,6.0,0,0,0.0,0.0,3.0,35056.0,190262.0,0,0,4,purpose_home_improvement,sub_grade_B3,home_ownership_MORTGAGE,22887.60,0.321074,6.4,-1.305838,0.234122,11.065838,-3.737064,0.440997,78.889879,False,False,0.157895,10.981934,2.571108,0.091059,0.388937,0.184251,1,0,0,0,0,325,325,25,5,5.944006,19.531001,1.585147,2.209072,9.759906,131.624466,11.211820,272.029410,10.464702,187.232476,12.156157,436.190325
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
29995,1002.03,13.29,30000,150000.0,10.0,36,0,2.0,1,0,0.0,0.0,3.0,3992.0,110852.0,0,0,29995,purpose_other,sub_grade_C1,home_ownership_OWN,36073.08,0.202436,6.7,5.273760,0.200000,21.306240,2.520024,0.623761,165.788449,False,False,0.130435,3.695067,0.739013,0.270631,1.353156,0.036012,1,0,0,0,1,0,0,0,0,6.909783,31.654857,2.587013,3.645545,10.308953,173.205081,11.918391,387.298335,8.292048,63.182276,11.615951,332.944440
29996,373.92,29.12,15000,60000.0,9.0,60,0,11.0,0,0,0.0,0.0,2.0,7482.0,70344.0,0,0,29996,purpose_debt_consolidation,sub_grade_D4,home_ownership_RENT,22435.20,0.495680,9.9,21.641600,0.250000,36.598400,1.345557,0.795663,389.388104,False,Fal

In [10]:
train['def'].value_counts()

0    25178
1     4822
Name: def, dtype: int64

In [11]:
25178/4822, 1/0.16

(5.221484861053505, 6.25)

In [ ]:
params = {
    "iterations": 1000,
    "depth": 3,
#     "learning_rate": 0.1,
#     "l2_leaf_reg": 0.1, # 50 
    ""
    "loss_function": "Logloss",
    "verbose": False,
    "eval_metric": 'AUC',
    "od_type": 'Iter',
    "od_wait": 100,
#     "class_weights": [1, 25178/4822]
#     "random_strength": 1,
}

columns = [
#     'term',
#     'acc_open_past_24mths', # 677 02
#     'funded_amnt',
#     'percent_per_year',
#     'additional_dti', #671
#     'num_accts_ever_120_pd',
#     'annual_inc', # 682 024,
#     'tot_hi_cred_lim',
#     'purpose', # 6839 035
#     'home_ownership', # 683 027
    
#     'avg_cur_bal', # 684 041
#     'mths_since_recent_inq',
#     'installment',
#     'funded_amnt_mod_100',


]

columns = targ_cols.copy()

remove_cols = [
    'is_dti_b_35',
    'is_not_sber_client',
    'is_first_loan',
    'tot_hi_cred_lim_log',
    'installment_sqrt',
    'installment_log',
    'funded_amnt_sqrt',
    'tot_hi_cred_lim_sqrt',
    'avg_cur_bal_sqrt',
    'is_emp_length_unknown',
    'dti_sqrt',
    'avg_cur_bal_log',
    'annual_inc_log',
    'annual_inc_sqrt',
    'dti_log',
    'funded_amnt_log',
    'even_installment',
    'is_common_amount',
    'delinq_amnt',
    'chargeoff_within_12_mths',
    'funded_amnt_mod_10',
    'even_salary',
    'home_ownership',
    'delinq_2yrs',
    'dti_ratio_max',
    'waste_to_installment_ratio',
    'funded_amnt_mod_1000',
    'num_tl_90g_dpd_24m',
    'funded_amnt_mod_500',
    'emp_length',
    'subgrade',
    'funded_amnt',
    'dti_ratio_min',
    'residual_dti',
    'mths_since_recent_inq',  
]

for col in remove_cols:
    columns.remove(col)

cat_features = [
    'purpose',
#     'home_ownership',
#     'subgrade',
#     'funded_amnt_mod_100',
]

skf = StratifiedKFold(n_splits=5, random_state=18)
oof_result = pd.Series([0] * train.shape[0])
result = [0] * test.shape[0]
scores = []
models = []
for ind_trn, ind_test in skf.split(X_train, y_train):
    X_train_df = X_train[columns].iloc[ind_trn]
    X_valid_df = X_train[columns].iloc[ind_test]
    y_train_df = y_train.iloc[ind_trn]
    y_valid_df = y_train.iloc[ind_test]

    eval_dataset = Pool(X_valid_df, y_valid_df, cat_features)

    model = CatBoostClassifier(
        **params,
#         per_float_feature_quantization=['7:border_count=1024']
    )

    model.fit(
        X_train_df,  
        y_train_df,
        cat_features,
        eval_set=eval_dataset,
        verbose=50,
        use_best_model=True,
    )
    result += model.predict_proba(test[columns])[:, 1:].flatten()/5
    scores.append(model.best_score_['validation']['AUC'])
    oof_result.loc[ind_test] = model.predict_proba(X_valid_df)[:, 1]
    models.append(model)


d:\kaggle\sber_plus_nsu\.venv\lib\site-packages\sklearn\model_selection\_split.py:297: FutureWarning: Setting a random_state has no effect since shuffle is False. This will raise an error in 0.24. You should leave random_state to its default (None), or set shuffle=True.
  FutureWarning


Learning rate set to 0.069474
0:	test: 0.6595782	best: 0.6595782 (0)	total: 85.6ms	remaining: 1m 25s
50:	test: 0.6960323	best: 0.6964262 (39)	total: 1.29s	remaining: 24s
100:	test: 0.6967916	best: 0.6968235 (99)	total: 3.21s	remaining: 28.6s
150:	test: 0.6975290	best: 0.6975290 (150)	total: 4.46s	remaining: 25.1s
200:	test: 0.6965520	best: 0.6975290 (150)	total: 5.9s	remaining: 23.5s
250:	test: 0.6954583	best: 0.6975290 (150)	total: 7.24s	remaining: 21.6s
Stopped by overfitting detector  (100 iterations wait)

bestTest = 0.6975290358
bestIteration = 150

Shrink model to first 151 iterations.
Learning rate set to 0.069474
0:	test: 0.6386913	best: 0.6386913 (0)	total: 83.4ms	remaining: 1m 23s
50:	test: 0.6850462	best: 0.6852877 (33)	total: 1.3s	remaining: 24.3s
100:	test: 0.6863290	best: 0.6863290 (100)	total: 2.57s	remaining: 22.9s
150:	test: 0.6872994	best: 0.6873676 (149)	total: 6.05s	remaining: 34s


In [ ]:
#depth 3, 0.07, no cw, l2 lr 20, 
np.mean(scores), np.std(scores), roc_auc_score(y_train, oof_result)

In [ ]:
plt.hist(oof_result[y_train == 1], bins=50, alpha=0.33, density='uniform', label='1')
plt.hist(oof_result[y_train == 0], bins=50, alpha=0.33, density='uniform', label='0')
plt.hist(result, bins=50, alpha=0.33, density='uniform', label='-1')
plt.legend()
plt.show()

In [ ]:
plt.hist(oof_result, bins=50, alpha=0.33, density='uniform', label='1')
# plt.hist(oof_result[y_train == 0], bins=50, alpha=0.33, density='uniform', label='0')
plt.hist(result, bins=50, alpha=0.33, density='uniform', label='-1')
plt.legend()
plt.show()

In [ ]:
test['prob'] = result

Выгрузка решения для коммита в соревновании

In [ ]:
test[['id', 'prob']].to_csv('cb_submission.csv', index=False)